# Import Library

In [166]:
import os
import cv2
import math
import numpy as np

# Data Path

In [167]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

train_path = "./images/train"
test_path = "./images/test"
model_path = "./face_recognizer_model.xml"

face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Train and Test Model

In [168]:
def menu1():

    person_name = os.listdir(train_path)
    face_list = []
    class_list = []

    for idx, name in enumerate(person_name):
        img_path = os.path.join(train_path, name)
        for img_name in os.listdir(img_path):
            full_img_path = os.path.join(img_path, img_name)
            img_gray = cv2.imread(full_img_path, cv2.IMREAD_GRAYSCALE)
            detected_faces = face_cascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5)

            for rect in detected_faces:
                x, y, w, h = rect
                face_img = img_gray[y:y+w, x:x+h]
                face_list.append(face_img)
                class_list.append(idx)

    face_recognizer.train(face_list, np.array(class_list))
    face_recognizer.save(model_path)

    correct_prediction = 0
    total_prediction = 0

    classes = os.listdir(train_path)

    for label, name in enumerate(classes):
        img_path = os.path.join(test_path, name)
        for img_name in os.listdir(img_path):
            full_img_path = os.path.join(img_path, img_name)
            img_gray = cv2.imread(full_img_path, cv2.IMREAD_GRAYSCALE)
            detected_faces = face_cascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5)

            for rect in detected_faces:
                x, y, w, h = rect
                face_img = img_gray[y:y+w, x:x+h]
                prediction, _ = face_recognizer.predict(face_img)
                if prediction == label:
                    correct_prediction += 1
                total_prediction += 1

    print (f"Model Accuracy : {correct_prediction / total_prediction * 100} %")


# Predict

In [169]:
def menu2(img_path):

    test_img = img_path
    person_name = os.listdir(train_path)

    face_recognizer.read(model_path)

    if not os.path.exists(model_path):
        print ("Model not exist yet, please train the model first")
        return
    
    img_bgr = cv2.imread(test_img)
    img_gry = cv2.cvtColor(img_bgr, cv2.COLOR_RGB2GRAY)

    detected_faces = face_cascade.detectMultiScale(img_gry, scaleFactor=1.2, minNeighbors=5)

    for rect in detected_faces:
        x, y, w, h = rect
        face_img = img_gry[y:y+w, x:x+h]
        label, confidence = face_recognizer.predict(face_img)

        confidence = math.floor(confidence * 100) / 100
        name = person_name[label]
        text = f"{name} : {confidence:.2f}%"

        cv2.rectangle(img_bgr, (x, y), (x+w, y+h), (0, 255, 255), 1)
        cv2.putText(img_bgr, text, (x, y-10), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1)
        cv2.imshow("Result", img_bgr)
        cv2.waitKey(0)


# Main Menu

In [170]:
def print_menu():
    print ("1. Train and Test Model")
    print ("2. Predict Image")
    print ("3. Exit")

In [171]:
def main():
    choice = -1
    while (True):
        print_menu()
        choice = int(input("Choose your menu: "))
        if choice == 1:
            menu1()
        elif choice == 2:
            path = input("Input Image Absolute Path: ")
            menu2(path)
        elif choice == 3:
            break
        else:
            print ("Input invalid please try again or 3 to quit")

    print ("Thank you for using this application :D")

In [172]:
main()

1. Train and Test Model
2. Predict Image
3. Exit
1. Train and Test Model
2. Predict Image
3. Exit
Thank you for using this application :D
